In [1]:
from pyflink.table import CsvTableSource, CsvTableSink, StreamTableEnvironment, EnvironmentSettings, Schema, DataTypes, WriteMode, ChangelogMode

In [2]:
from pyflink.datastream import StreamExecutionEnvironment

In [3]:
from pyflink.table.expressions import col as col_fk

In [4]:
from pyflink.common.typeinfo import Types, TypeInformation

In [5]:
stream_env = StreamExecutionEnvironment.get_execution_environment()

In [6]:
env_set = EnvironmentSettings.new_instance()\
                            .in_streaming_mode()\
                            .build()

In [7]:
stream_tbl_env = StreamTableEnvironment.create(stream_execution_environment=stream_env, 
                                               environment_settings=env_set)

In [8]:
# parallelism should be 1 because join is by default done by parallelism creating multiple file

In [9]:
stream_tbl_env.get_config().get_configuration().set_string("parallelism.default", "1")

In [10]:
stream_env.set_parallelism(1)

In [11]:
csv_tbl_sch_name_1 = ['seller_id', 'product', 'quantity', 'product_price', 'sales_date']
csv_tbl_sch_type_1 = [DataTypes.STRING(), DataTypes.STRING(), DataTypes.BIGINT(), DataTypes.DOUBLE(), DataTypes.DATE()]

In [12]:
csv_tbl_sch_name_2 = ['id', 'city', 'state']
csv_tbl_sch_type_2 = [DataTypes.STRING(), DataTypes.STRING(), DataTypes.STRING()]

In [13]:
csv_tbl_1 = CsvTableSource(source_path='locale-sales.csv', 
                           field_names=csv_tbl_sch_name_1, 
                           field_types=csv_tbl_sch_type_1, 
                           ignore_first_line=True, 
                           empty_column_as_null=True)

In [14]:
csv_tbl_2 = CsvTableSource(source_path='sellers.csv', 
                           field_names=csv_tbl_sch_name_2, 
                           field_types=csv_tbl_sch_type_2, 
                           ignore_first_line=True, 
                           empty_column_as_null=True)

In [15]:
csv_tbl_load_1 = stream_tbl_env.from_table_source(csv_tbl_1)

In [16]:
csv_tbl_load_2 = stream_tbl_env.from_table_source(csv_tbl_2)

In [17]:
csv_tbl_load_join = csv_tbl_load_1.left_outer_join(right=csv_tbl_load_2, join_predicate=(col_fk('seller_id') == col_fk('id')))

In [18]:
csv_ds_load_join = stream_tbl_env.to_changelog_stream(table=csv_tbl_load_join,
                                                      changelog_mode=ChangelogMode.all())

In [19]:
# with csv_ds_load_join.execute_and_collect() as results:
#     for result in results:
#         print(result)

In [20]:
csv_tbl_load_join = stream_tbl_env.from_data_stream(csv_ds_load_join)

In [21]:
# with csv_tbl_load_join.execute().collect() as results:
#     for result in results:
#         print(result)

In [22]:
csv_tbl_sch_name_join = csv_tbl_load_join.get_schema().get_field_names()
csv_tbl_sch_type_join = csv_tbl_load_join.get_schema().get_field_data_types()

In [23]:
csv_tbl_sink_join = CsvTableSink(field_names=csv_tbl_sch_name_join,
                                 field_types=csv_tbl_sch_type_join,
                                 path='./CSV/seller_join.csv',
                                 field_delimiter=',',
                                 num_files=-1,
                                 write_mode=WriteMode.OVERWRITE)

In [24]:
stream_tbl_env.register_table_sink('csv_tbl_sink_join', csv_tbl_sink_join)

In [25]:
csv_tbl_load_join.execute_insert('csv_tbl_sink_join')